In [1]:
#The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents,
#partitioned (nearly) evenly across 20 different newsgroups. To the best of our knowledge,
#it was originally collected by Ken Lang, probably for his paper 
#“Newsweeder: Learning to filter netnews,” though he does not explicitly mention this
#collection.
#The 20 newsgroups collection has become a popular data set for experiments
#in text applications of machine learning techniques, such as text classification 
#and text clustering.

In [2]:
# selecting labels 
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [3]:
import sklearn
from sklearn.datasets import fetch_20newsgroups

dataset = sklearn.datasets.load_files("../text_analytics/data/twenty_newsgroups/20news-bydate-train/", categories=categories, load_content=True, shuffle=True, encoding='UTF-8', decode_error='replace', random_state=0)

In [4]:
# You can explore the data a bit
print dataset.target_names
print("\n".join(dataset.data[0].split("\n")[:3]))
print(dataset.target_names[dataset.target[0]])


['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul Checkman)
Subject: Re: Is MSG sensitivity superstition?
Article-I.D.: news.C5wI4F.Dt
sci.med


In [5]:
# Tokenisation
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset.data)



In [12]:
# How many words in total does your dictionary have?
print len(count_vect.get_feature_names())

# Which words are the most common?
from collections import defaultdict, Counter

features = count_vect.get_feature_names()

counter = Counter()
for x in X_train_counts:
    xc = x.tocsr()
    xs = zip(xc.indices,xc.data)
    for a,b in xs:
        counter[features[a]]+=b
print counter
# How many times does GPU appear?
print counter['gpu']

35786
Counter({u'the': 29825, u'of': 17660, u'to': 17222, u'and': 13017, u'is': 12343, u'in': 11226, u'that': 10723, u'it': 8447, u'you': 6157, u'for': 5962, u'not': 5285, u'this': 5063, u'be': 4898, u'from': 4877, u'are': 4769, u'edu': 4675, u'have': 4218, u'as': 4015, u'on': 3709, u'with': 3655, u'but': 3453, u'or': 3432, u'if': 3208, u'was': 3020, u'can': 2818, u'we': 2674, u'there': 2556, u'by': 2548, u'what': 2472, u'subject': 2464, u'they': 2448, u'an': 2393, u'lines': 2353, u'one': 2312, u'he': 2232, u'god': 2216, u'do': 2193, u'organization': 2192, u'at': 2191, u're': 2186, u'com': 2172, u'about': 2148, u'my': 2143, u'all': 2093, u'would': 2000, u'will': 1925, u'so': 1912, u'some': 1802, u'your': 1793, u'no': 1791, u'who': 1742, u'has': 1671, u'any': 1669, u'writes': 1645, u'people': 1604, u'which': 1599, u'me': 1543, u'article': 1453, u'don': 1400, u'his': 1318, u'more': 1286, u'other': 1278, u'know': 1230, u'when': 1215, u'think': 1213, u'their': 1184, u'like': 1177, u'univer

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
# Feature representation
# TF
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

#TF_IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [16]:
# Classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, dataset.target)

In [17]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, dataset.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [18]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

dataset_test = sklearn.datasets.load_files("../text_analytics/data/twenty_newsgroups/20news-bydate-test/", categories=categories, load_content=True, shuffle=True, encoding='UTF-8', decode_error='replace', random_state=42)
docs_test = dataset_test.data
clf = MultinomialNB()

#X_train_counts = count_vect.fit_transform(dataset.data)
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(dataset.data)

X_test_counts = count_vect.transform(docs_test)

print len(count_vect.get_feature_names())

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

text_clf = clf.fit(X_train_tfidf, dataset.target)
predicted = text_clf.predict(X_test_tfidf)
np.mean(predicted == dataset_test.target)

35480


0.88948069241011984

In [25]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
#What happens if you change the classifier?
clf = AdaBoostClassifier(n_estimators=100)

# Using the Tfidf Vectorizer means you don't need to use the counter first
# :-)

#What happens if you exclude stop words or limit the size of the dictionary?
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(dataset.data)
X_test_tfidf = tfidf_vectorizer.transform(docs_test)

text_clf = clf.fit(X_train_tfidf, dataset.target)
predicted = text_clf.predict(X_test_tfidf)
np.mean(predicted == dataset_test.target)



0.73235685752330226